In [2]:
import os
import cv2

input_directory = "/content/drive/MyDrive/luffy"
output_directory = "/content/drive/MyDrive/Loras/luffy_gear_5/dataset"

In [ ]:
## Preprocessing Images

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for filename in os.listdir(input_directory):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".webp"):
        img_path = os.path.join(input_directory, filename)
        print("Processing image:", img_path)  # Print the image path for debugging
        image = cv2.imread(img_path)

        if image is None:
            print("Failed to load image:", img_path)
            continue

        resized_image = cv2.resize(image, (512, 512))
        output_path = os.path.join(output_directory, filename)
        cv2.imwrite(output_path, resized_image)

        print(f"Resized and saved {filename} to {output_path}")

print("Image resizing and storing complete.")

### ⭕ Disclaimer
The purpose of this document is to research bleeding-edge technologies in the field of machine learning inference.  
Please read and follow the [Google Colab guidelines](https://research.google.com/colaboratory/faq.html) and its [Terms of Service](https://research.google.com/colaboratory/tos_v3.html).

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 🏠 **Homepage** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab) | | |
| 📊 **Dataset Maker** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |

In [1]:
import os
from IPython import get_ipython
from IPython.display import display, Markdown

COLAB = True

if COLAB:
  from google.colab.output import clear as clear_output
else:
  from IPython.display import clear_output

#@title ## 🚩 Start Here

#@markdown ### 1️⃣ Setup
#@markdown This cell will load some requirements and create the necessary folders in your Google Drive. <p>
#@markdown Your project name can't contain spaces but it can contain a single / to make a subfolder in your dataset.
project_name = "luffy_gear_5" #@param {type:"string"}
project_name = project_name.strip()
#@markdown The folder structure doesn't matter and is purely for comfort. Make sure to always pick the same one. I like organizing by project.
folder_structure = "Organize by project (MyDrive/Loras/project_name/dataset)" #@param ["Organize by category (MyDrive/lora_training/datasets/project_name)", "Organize by project (MyDrive/Loras/project_name/dataset)"]

if not project_name or any(c in project_name for c in " .()\"'\\") or project_name.count("/") > 1:
  print("Please write a valid project_name.")
else:
  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

  project_base = project_name if "/" not in project_name else project_name[:project_name.rfind("/")]
  project_subfolder = project_name if "/" not in project_name else project_name[project_name.rfind("/")+1:]

  root_dir = "/content" if COLAB else "~/Loras"
  deps_dir = os.path.join(root_dir, "deps")

  if "/Loras" in folder_structure:
    main_dir      = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
    config_folder = os.path.join(main_dir, project_base)
    images_folder = os.path.join(main_dir, project_base, "dataset")
    if "/" in project_name:
      images_folder = os.path.join(images_folder, project_subfolder)
  else:
    main_dir      = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
    config_folder = os.path.join(main_dir, "config", project_name)
    images_folder = os.path.join(main_dir, "datasets", project_name)

  for dir in [main_dir, deps_dir, images_folder, config_folder]:
    os.makedirs(dir, exist_ok=True)

  print(f"✅ Project {project_name} is ready!")
  step1_installed_flag = True


📂 Connecting to Google Drive...
Mounted at /content/drive
✅ Project luffy_gear_5 is ready!


In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@markdown ### 3️⃣ Curate your images
#@markdown We will find duplicate images with the FiftyOne AI, and mark them with `delete`. <p>
#@markdown Then, an interactive area will appear below this cell that lets you visualize all your images and manually mark with `delete` to the ones you don't like. <p>
#@markdown If the interactive area appears blank for over a minute, try enabling cookies and removing tracking protection for the Google Colab website, as they may break it.
#@markdown Regardless, you can save your changes by sending Enter in the input box above the interactive area.<p>
#@markdown This is how similar 2 images must be to be marked for deletion. I recommend 0.97 to 0.99:
similarity_threshold = 0.985 #@param {type:"number"}


os.chdir(root_dir)
model_name = "clip-vit-base32-torch"
supported_types = (".png", ".jpg", ".jpeg")
img_count = len(os.listdir(images_folder))
batch_size = min(250, img_count)

if "step3_installed_flag" not in globals():
  print("🏭 Installing dependencies...\n")
  !pip -q install fiftyone ftfy
  !pip -q install fiftyone-db-ubuntu2204
  if not get_ipython().__dict__['user_ns']['_exit_code']:
    clear_output()
    step3_installed_flag = True
  else:
    print("❌ Error installing dependencies, attempting to continue anyway...")

import numpy as np
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
from sklearn.metrics.pairwise import cosine_similarity

non_images = [f for f in os.listdir(images_folder) if not f.lower().endswith(supported_types)]
if non_images:
  print(f"💥 Error: Found non-image file {non_images[0]} - This program doesn't allow it. Sorry! Use the Extras at the bottom to clean the folder.")
elif img_count == 0:
  print(f"💥 Error: No images found in {images_folder}")
else:
  print("\n💿 Analyzing dataset...\n")
  dataset = fo.Dataset.from_dir(images_folder, dataset_type=fo.types.ImageDirectory)
  model = foz.load_zoo_model(model_name)
  embeddings = dataset.compute_embeddings(model, batch_size=batch_size)

  batch_embeddings = np.array_split(embeddings, batch_size)
  similarity_matrices = []
  max_size_x = max(array.shape[0] for array in batch_embeddings)
  max_size_y = max(array.shape[1] for array in batch_embeddings)

  for i, batch_embedding in enumerate(batch_embeddings):
    similarity = cosine_similarity(batch_embedding)
    #Pad 0 for np.concatenate
    padded_array = np.zeros((max_size_x, max_size_y))
    padded_array[0:similarity.shape[0], 0:similarity.shape[1]] = similarity
    similarity_matrices.append(padded_array)

  similarity_matrix = np.concatenate(similarity_matrices, axis=0)
  similarity_matrix = similarity_matrix[0:embeddings.shape[0], 0:embeddings.shape[0]]

  similarity_matrix = cosine_similarity(embeddings)
  similarity_matrix -= np.identity(len(similarity_matrix))

  dataset.match(F("max_similarity") > similarity_threshold)
  dataset.tags = ["delete", "has_duplicates"]

  id_map = [s.id for s in dataset.select_fields(["id"])]
  samples_to_remove = set()
  samples_to_keep = set()

  for idx, sample in enumerate(dataset):
    if sample.id not in samples_to_remove:
      # Keep the first instance of two duplicates
      samples_to_keep.add(sample.id)

      dup_idxs = np.where(similarity_matrix[idx] > similarity_threshold)[0]
      for dup in dup_idxs:
          # We kept the first instance so remove all other duplicates
          samples_to_remove.add(id_map[dup])

      if len(dup_idxs) > 0:
          sample.tags.append("has_duplicates")
          sample.save()
    else:
      sample.tags.append("delete")
      sample.save()

  clear_output()

  sidebar_groups = fo.DatasetAppConfig.default_sidebar_groups(dataset)
  for group in sidebar_groups[1:]:
    group.expanded = False
  dataset.app_config.sidebar_groups = sidebar_groups
  dataset.save()
  session = fo.launch_app(dataset)

  print("❗ Wait a minute for the session to load. If it doesn't, read above.")
  print("❗ When it's ready, you'll see a grid of your images.")
  print("❗ On the left side enable \"sample tags\" to visualize the images marked for deletion.")
  print("❗ You can mark your own images with the \"delete\" label by selecting them and pressing the tag icon at the top.")
  input("⭕ When you're done, enter something here to save your changes: ")

  print("💾 Saving...")

  kys = [s for s in dataset if "delete" in s.tags]
  dataset.remove_samples(kys)
  previous_folder = images_folder[:images_folder.rfind("/")]
  dataset.export(export_dir=os.path.join(images_folder, project_subfolder), dataset_type=fo.types.ImageDirectory)

  temp_suffix = "_temp"
  !mv {images_folder} {images_folder}{temp_suffix}
  !mv {images_folder}{temp_suffix}/{project_subfolder} {images_folder}
  !rm -r {images_folder}{temp_suffix}

  session.refresh()
  fo.close_app()
  clear_output()

  print(f"\n✅ Removed {len(kys)} images from dataset. You now have {len(os.listdir(images_folder))} images.")



✅ Removed 0 images from dataset. You now have 456 images.


In [4]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@markdown ### 4️⃣ Tag your images
#@markdown We will be using AI to automatically tag your images, specifically [Waifu Diffusion](https://huggingface.co/SmilingWolf/wd-v1-4-swinv2-tagger-v2) in the case of anime and [BLIP](https://huggingface.co/spaces/Salesforce/BLIP) in the case of photos.
#@markdown Giving tags/captions to your images allows for much better training. This process should take a couple minutes. <p>
method = "Anime tags" #@param ["Anime tags", "Photo captions"]
#@markdown **Anime:** The threshold is the minimum level of confidence the tagger must have in order to include a tag. Lower threshold = More tags. Recommended 0.35 to 0.5
tag_threshold = 0.35 #@param {type:"slider", min:0.0, max:1.0, step:0.01}
blacklist_tags = "bangs, breasts, multicolored hair, two-tone hair, gradient hair, virtual youtuber, official alternate costume, official alternate hairstyle, official alternate hair length, alternate costume, alternate hairstyle, alternate hair length, alternate hair color" #@param {type:"string"}
#@markdown **Photos:** The minimum and maximum length of tokens/words in each caption.
caption_min = 10 #@param {type:"number"}
caption_max = 75 #@param {type:"number"}

%env PYTHONPATH=/env/python
os.chdir(root_dir)
kohya = "/content/kohya-trainer"
if not os.path.exists(kohya):
  !git clone https://github.com/kohya-ss/sd-scripts {kohya}
  os.chdir(kohya)
  !git reset --hard 5050971ac687dca70ba0486a583d283e8ae324e2
  os.chdir(root_dir)

if "tags" in method:
  if "step4a_installed_flag" not in globals():
    print("\n🏭 Installing dependencies...\n")
    !pip -q install tensorflow==2.12.0 huggingface-hub==0.12.0 accelerate==0.15.0 transformers==4.26.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6 torchvision albumentations
    if not get_ipython().__dict__['user_ns']['_exit_code']:
      clear_output()
      step4a_installed_flag = True
    else:
      print("❌ Error installing dependencies, trying to continue anyway...")

  print("\n🚶‍♂️ Launching program...\n")

  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
  %env PYTHONPATH={kohya}
  !python {kohya}/finetune/tag_images_by_wd14_tagger.py \
    {images_folder} \
    --repo_id=SmilingWolf/wd-v1-4-swinv2-tagger-v2 \
    --model_dir={root_dir} \
    --thresh={tag_threshold} \
    --batch_size=8 \
    --caption_extension=.txt \
    --force_download

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    print("removing underscores and blacklist...")
    blacklisted_tags = [t.strip() for t in blacklist_tags.split(",")]
    from collections import Counter
    top_tags = Counter()
    for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:
      with open(os.path.join(images_folder, txt), 'r') as f:
        tags = [t.strip() for t in f.read().split(",")]
        tags = [t.replace("_", " ") if len(t) > 3 else t for t in tags]
        tags = [t for t in tags if t not in blacklisted_tags]
      top_tags.update(tags)
      with open(os.path.join(images_folder, txt), 'w') as f:
        f.write(", ".join(tags))

    %env PYTHONPATH=/env/python
    clear_output()
    print(f"📊 Tagging complete. Here are the top 50 tags in your dataset:")
    print("\n".join(f"{k} ({v})" for k, v in top_tags.most_common(50)))


else: # Photos
  if "step4b_installed_flag" not in globals():
    print("\n🏭 Installing dependencies...\n")
    !pip -q install timm==0.6.12 fairscale==0.4.13 transformers==4.26.0 requests==2.28.2 accelerate==0.15.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6
    if not get_ipython().__dict__['user_ns']['_exit_code']:
      clear_output()
      step4b_installed_flag = True
    else:
      print("❌ Error installing dependencies, trying to continue anyway...")

  print("\n🚶‍♂️ Launching program...\n")

  os.chdir(kohya)
  %env PYTHONPATH={kohya}
  !python {kohya}/finetune/make_captions.py \
    {images_folder} \
    --beam_search \
    --max_data_loader_n_workers=2 \
    --batch_size=8 \
    --min_length={caption_min} \
    --max_length={caption_max} \
    --caption_extension=.txt

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    import random
    captions = [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]
    sample = []
    for txt in random.sample(captions, min(10, len(captions))):
      with open(os.path.join(images_folder, txt), 'r') as f:
        sample.append(f.read())

    os.chdir(root_dir)
    %env PYTHONPATH=/env/python
    clear_output()
    print(f"📊 Captioning complete. Here are {len(sample)} example captions from your dataset:")
    print("".join(sample))


📊 Tagging complete. Here are the top 50 tags in your dataset:
solo (21)
smile (21)
1boy (21)
male focus (21)
white hair (20)
grin (19)
scar (19)
looking at viewer (13)
teeth (10)
open clothes (10)
scar on chest (9)
scar on face (8)
straw hat (7)
shirt (6)
open shirt (6)
shorts (5)
pectorals (5)
upper body (5)
red eyes (3)
close-up (3)
style parody (3)
fire (3)
abs (3)
long hair (3)
hat (3)
pink eyes (2)
glowing (2)
white shorts (2)
open mouth (2)
weapon (2)
sword (2)
collarbone (2)
portrait (2)
yellow eyes (2)
closed eyes (2)
full body (2)
sandals (2)
clenched hands (2)
blonde hair (2)
dark skin (2)
dark-skinned male (2)
evil smile (1)
orange eyes (1)
magic (1)
blue fire (1)
simple background (1)
outstretched arm (1)
black background (1)
clenched hand (1)
incoming attack (1)


In [5]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@markdown ### 5️⃣ Curate your tags
#@markdown Modify your dataset's tags. You can run this cell multiple times with different parameters. <p>

#@markdown Put an activation tag at the start of every text file. This is useful to make learning better and activate your Lora easier. Set `keep_tokens` to 1 when training.<p>
#@markdown Common tags that are removed such as hair color, etc. will be "absorbed" by your activation tag.
global_activation_tag = "Gear5" #@param {type:"string"}
remove_tags = "" #@param {type:"string"}
#@markdown &nbsp;

#@markdown In this advanced section, you can search text files containing matching tags, and replace them with less/more/different tags. If you select the checkbox below, any extra tags will be put at the start of the file, letting you assign different activation tags to different parts of your dataset. Still, you may want a more advanced tool for this.
search_tags = "" #@param {type:"string"}
replace_with = "" #@param {type:"string"}
search_mode = "OR" #@param ["OR", "AND"]
new_becomes_activation_tag = True #@param {type:"boolean"}
#@markdown These may be useful sometimes. Will remove existing activation tags, be careful.
sort_alphabetically = False #@param {type:"boolean"}
remove_duplicates = False #@param {type:"boolean"}

def split_tags(tagstr):
  return [s.strip() for s in tagstr.split(",") if s.strip()]

activation_tag_list = split_tags(global_activation_tag)
remove_tags_list = split_tags(remove_tags)
search_tags_list = split_tags(search_tags)
replace_with_list = split_tags(replace_with)
replace_new_list = [t for t in replace_with_list if t not in search_tags_list]

replace_with_list = [t for t in replace_with_list if t not in replace_new_list]
replace_new_list.reverse()
activation_tag_list.reverse()

remove_count = 0
replace_count = 0

for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:

  with open(os.path.join(images_folder, txt), 'r') as f:
    tags = [s.strip() for s in f.read().split(",")]

  if remove_duplicates:
    tags = list(set(tags))
  if sort_alphabetically:
    tags.sort()

  for rem in remove_tags_list:
    if rem in tags:
      remove_count += 1
      tags.remove(rem)

  if "AND" in search_mode and all(r in tags for r in search_tags_list) \
      or "OR" in search_mode and any(r in tags for r in search_tags_list):
    replace_count += 1
    for rem in search_tags_list:
      if rem in tags:
        tags.remove(rem)
    for add in replace_with_list:
      if add not in tags:
        tags.append(add)
    for new in replace_new_list:
      if new_becomes_activation_tag:
        if new in tags:
          tags.remove(new)
        tags.insert(0, new)
      else:
        if new not in tags:
          tags.append(new)

  for act in activation_tag_list:
    if act in tags:
      tags.remove(act)
    tags.insert(0, act)

  with open(os.path.join(images_folder, txt), 'w') as f:
    f.write(", ".join(tags))

if global_activation_tag:
  print(f"\n📎 Applied new activation tag(s): {', '.join(activation_tag_list)}")
if remove_tags:
  print(f"\n🚮 Removed {remove_count} tags.")
if search_tags:
  print(f"\n💫 Replaced in {replace_count} files.")
print("\n✅ Done! Check your updated tags in the Extras below.")



📎 Applied new activation tag(s): Gear5

✅ Done! Check your updated tags in the Extras below.


In [ ]:
#@markdown ### 6️⃣ Ready
#@markdown You should be ready to [train your Lora](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb)!

from IPython.display import Markdown, display
display(Markdown(f"### 🦀 [Click here to open the Lora trainer](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb)"))


### 🦀 [Click here to open the Lora trainer](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb)

## *️⃣ Extras

In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@markdown ### 📈 Analyze Tags
#@markdown Perhaps you need another look at your dataset.
show_top_tags = 50 #@param {type:"number"}

from collections import Counter
top_tags = Counter()

for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:
  with open(os.path.join(images_folder, txt), 'r') as f:
    top_tags.update([s.strip() for s in f.read().split(",")])

top_tags = Counter(top_tags)
print(f"📊 Top {show_top_tags} tags:")
for k, v in top_tags.most_common(show_top_tags):
  print(f"{k} ({v})")

📊 Top 50 tags:
Naruto_Style (1221)
solo (1079)
male focus (892)
1boy (857)
looking at viewer (621)
closed mouth (582)
upper body (484)
black eyes (477)
black hair (437)
parody (334)
brown hair (322)
forehead protector (321)
smile (299)
short hair (276)
1girl (265)
portrait (261)
outdoors (242)
long hair (198)
facial hair (195)
sky (188)
konohagakure symbol (187)
headband (186)
day (169)
ninja (164)
frown (163)
meme (160)
blurry (150)
cloud (131)
japanese clothes (129)
brown eyes (125)
blue sky (116)
tree (113)
beard (112)
hat (108)
shirt (107)
teeth (102)
no humans (93)
multiple boys (90)
collarbone (89)
simple background (88)
spiked hair (85)
indoors (85)
blonde hair (84)
mustache (83)
vest (82)
grey hair (81)
open mouth (79)
weapon (79)
blue eyes (76)
jewelry (73)


In [ ]:
#@markdown ### 📂 Unzip dataset
#@markdown It's much slower to upload individual files to your Drive, so you may want to upload a zip if you have your dataset in your computer.
zip = "/content/drive/MyDrive/Loras/example.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/Loras/example/dataset" #@param {type:"string"}

import os, zipfile

if not os.path.exists('/content/drive'):
  from google.colab import drive
  print("📂 Connecting to Google Drive...")
  drive.mount('/content/drive')

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip, 'r') as f:
  f.extractall(extract_to)

print("✅ Done")


In [6]:
#@markdown ### 🔢 Count datasets
#@markdown Google Drive makes it impossible to count the files in a folder, so this will show you the file counts in all folders and subfolders.
folder = "/content/drive/MyDrive/Loras" #@param {type:"string"}

import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    print("📂 Connecting to Google Drive...\n")
    drive.mount('/content/drive')

tree = {}
exclude = ("_logs", "/output")
for i, (root, dirs, files) in enumerate(os.walk(folder, topdown=True)):
  dirs[:] = [d for d in dirs if all(ex not in d for ex in exclude)]
  images = len([f for f in files if f.lower().endswith((".png", ".jpg", ".jpeg"))])
  captions = len([f for f in files if f.lower().endswith(".txt")])
  others = len(files) - images - captions
  path = root[folder.rfind("/")+1:]
  tree[path] = None if not images else f"{images:>4} images | {captions:>4} captions |"
  if tree[path] and others:
    tree[path] += f" {others:>4} other files"

pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))


📁Loras/narutoda/dataset                        | 1629 images | 1620 captions |  802 other files
📁Loras/Naruto_real_anime_ds/dataset            |  548 images |  542 captions |
📁Loras/Jiraiya_sensei/dataset                  |  122 images |  112 captions |
📁Loras/Itachi_anime/dataset                    |   50 images |   49 captions |
📁Loras/Kakashi_anime/dataset                   |   38 images |   38 captions |
📁Loras/Madara_anime/dataset                    |   82 images |   82 captions |
📁Loras/Naruto_Shippuden/dataset                |   71 images |   71 captions |
📁Loras/Naruto_an/dataset                       |   22 images |   22 captions |
📁Loras/Naruto_fanart_anylora/dataset           |  456 images |  455 captions |
📁Loras/Images                                  | 1221 images |    0 captions |
📁Loras/Naruto_anime_anylora/dataset            | 1221 images | 1221 captions |
📁Loras/luffy_gear_5/dataset                    |   25 images |   21 captions |    5 other files


In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@markdown ### 🚮 Clean folder
#@markdown Careful! Deletes all non-image files in the project folder.

!find {images_folder} -type f ! \( -name '*.png' -o -name '*.jpg' -o -name '*.jpeg' \) -delete
